In [ ]:
%%bash
pip install pillow numpy

In [ ]:
from pathlib import Path
import numpy as np
from PIL import Image, ImageDraw, ImageFont

OUT = Path("skin_before_after_captioned.jpg")            # final result

# LOAD IMAGE & FIND THE Two photo REGIONS (simple threshold + split)
img  = Image.open(Path("~/Downloads/procell1.jpg").expanduser())
arr = np.array(img.convert("L"))
print(f"Image size is {arr.shape}")
ys, xs = np.where(arr < 200)                # “not-white” ≈ photo content
print(list(map(int, [ys.min(), ys.max(), xs.min(), xs.max()])))
# img

In [ ]:
mid_y  = (ys.min() + ys.max()) // 2         # halfway down the collage

# restrict search to the right half where the photos live
x_lo, x_hi = 480, xs.max()

def bbox(idx):
    inset = 1
    y, x = ys[idx], xs[idx]
    return (x.min() + inset, y.min() + inset, x.max() - inset, y.max() - inset)

before_box = bbox((ys < mid_y)  & (xs > x_lo) & (xs < x_hi))
after_box  = bbox((ys >= mid_y) & (xs > x_lo) & (xs < x_hi))

before_img = img.crop(before_box)
after_img  = img.crop(after_box)

In [ ]:
# 2) STITCH PHOTOS SIDE-BY-SIDE (no text)
canvas_w = before_img.width + after_img.width
canvas_h = min(before_img.height, after_img.height)
canvas_h = canvas_w * 9 // 16
offset = (canvas_h - min(before_img.height, after_img.height)) // 2
canvas = Image.new("RGB", (canvas_w, canvas_h), "white")
canvas.paste(before_img, (0, offset + 50))
canvas.paste(after_img,  (before_img.width, offset - 50))

FONT_SIZE  = 64
draw = ImageDraw.Draw(canvas)
font = ImageFont.truetype("/System/Library/Fonts/Helvetica.ttc", FONT_SIZE)

y_text  = canvas_h - FONT_SIZE // 2
draw.text((canvas_w // 4,       y_text), "before", anchor="mm", font=font, fill="white")
draw.text((canvas_w * 3 // 4,   y_text), "after",  anchor="mm", font=font, fill="white")

canvas

In [ ]:
canvas.save(OUT)
print(OUT.resolve())